Deep Learning
=============

Assignment 6
------------

After training a skip-gram model in `5_word2vec.ipynb`, the goal of this notebook is to train a LSTM character model over [Text8](http://mattmahoney.net/dc/textdata) data.

In [2]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import os
import numpy as np
import random
import string
import tensorflow as tf
import zipfile
from six.moves import range
from six.moves.urllib.request import urlretrieve

In [3]:
url = 'http://mattmahoney.net/dc/'

def maybe_download(filename, url_path, expected_bytes):
  """Download a file if not present, and make sure it's the right size."""
  if not os.path.exists(filename):
    filename, _ = urlretrieve(url + url_path, filename)
  statinfo = os.stat(filename)
  if statinfo.st_size == expected_bytes:
    print('Found and verified %s' % filename)
  else:
    print(statinfo.st_size)
    raise Exception(
      'Failed to verify ' + filename + '. Can you get to it with a browser?')
  return filename

filename = maybe_download('../data/text8.zip', 'text8.zip', 31344016)

Found and verified ../data/text8.zip


In [4]:
def read_data(filename):
  with zipfile.ZipFile(filename) as f:
    name = f.namelist()[0]
    data = tf.compat.as_str(f.read(name))
  return data
  
text = read_data(filename)
print('Data size %d' % len(text))

Data size 100000000


Create a small validation set.

In [5]:
valid_size = 1000
valid_text = text[:valid_size]
train_text = text[valid_size:]
train_size = len(train_text)
print(train_size, train_text[:64])
print(valid_size, valid_text[:64])

99999000 ons anarchists advocate social relations based upon voluntary as
1000  anarchism originated as a term of abuse first used against earl


Utility functions to map characters to vocabulary IDs and back.

In [6]:
vocabulary_size = len(string.ascii_lowercase) + 1 # [a-z] + ' '
first_letter = ord(string.ascii_lowercase[0])

def char2id(char):
  if char in string.ascii_lowercase:
    return ord(char) - first_letter + 1
  elif char == ' ':
    return 0
  else:
    print('Unexpected character: %s' % char)
    return 0
  
def id2char(dictid):
  if dictid > 0:
    return chr(dictid + first_letter - 1)
  else:
    return ' '

print(char2id('a'), char2id('z'), char2id(' '), char2id('ï'))
print('id2char...')
print('(', id2char(1),',', id2char(26),',', id2char(0),')')
print('vocabulary_size: ', vocabulary_size)

Unexpected character: ï
1 26 0 0
id2char...
( a , z ,   )
vocabulary_size:  27


Function to generate a training batch for the LSTM model.

In [7]:
batch_size=64
num_unrollings=10

class BatchGenerator(object):
  def __init__(self, text, batch_size, num_unrollings):
    self._text = text
    self._text_size = len(text)
    self._batch_size = batch_size
    self._num_unrollings = num_unrollings
    segment = self._text_size // batch_size
    self._cursor = [ offset * segment for offset in range(batch_size)]
    self._last_batch = self._next_batch()
  
  def _next_batch(self):
    """Generate a single batch from the current cursor position in the data."""
    batch = np.zeros(shape=(self._batch_size, vocabulary_size), dtype=np.float)
    for b in range(self._batch_size):
      batch[b, char2id(self._text[self._cursor[b]])] = 1.0
      self._cursor[b] = (self._cursor[b] + 1) % self._text_size
    return batch
  
  def next(self):
    """Generate the next array of batches from the data. The array consists of
    the last batch of the previous array, followed by num_unrollings new ones.
    """
    batches = [self._last_batch]
    for step in range(self._num_unrollings):
      batches.append(self._next_batch())
    self._last_batch = batches[-1]
    return batches

def characters(probabilities):
  """Turn a 1-hot encoding or a probability distribution over the possible
  characters back into its (most likely) character representation."""
  return [id2char(c) for c in np.argmax(probabilities, 1)]

def batches2string(batches):
  """Convert a sequence of batches back into their (most likely) string
  representation."""
  s = [''] * batches[0].shape[0] # this is just batch_size
  for b in batches:
    s = [''.join(x) for x in zip(s, characters(b))]
  return s

train_batches = BatchGenerator(train_text, batch_size, num_unrollings)
valid_batches = BatchGenerator(valid_text, 1, 1)

# Below code is just for testing
first_batch = train_batches.next()
print('len(first_batch): ', len(first_batch))
print('first_batch[0].shape', first_batch[0].shape)
print('len(first_batch[0]:', len(first_batch[0]))
print('len(first_batch[0][0]:', len(first_batch[0][0]))

s = [''] * first_batch[0].shape[0] # this is just batch_size (==64)
print('first letter of each element in the first batch:', [''.join(x) for x in zip(s, characters(first_batch[0]))])

print('\nbatches to strings:')
print(batches2string(first_batch))
print(batches2string(train_batches.next()))
print(batches2string(valid_batches.next()))
print(batches2string(valid_batches.next()))

len(first_batch):  11
first_batch[0].shape (64, 27)
len(first_batch[0]: 64
len(first_batch[0][0]: 27
first letter of each element in the first batch: ['o', 'w', 'l', ' ', 'm', 'h', 'y', 'a', 't', 'm', 'n', 'h', 'e', 'e', 'o', 'y', 'o', 'a', ' ', 'a', 'i', ' ', 't', 'd', 'f', 'a', 'e', 'e', 'a', 'r', 'i', 'o', 'a', 'g', 'i', 'r', 'c', 'a', ' ', 'm', 't', 'u', 'e', 'o', 'o', 's', 'k', 'e', 'w', 'e', 't', 'e', ' ', 'i', 't', 'd', 't', 'e', 'f', 'd', 't', 'a', 'a', 's']

batches to strings:
['ons anarchi', 'when milita', 'lleria arch', ' abbeys and', 'married urr', 'hel and ric', 'y and litur', 'ay opened f', 'tion from t', 'migration t', 'new york ot', 'he boeing s', 'e listed wi', 'eber has pr', 'o be made t', 'yer who rec', 'ore signifi', 'a fierce cr', ' two six ei', 'aristotle s', 'ity can be ', ' and intrac', 'tion of the', 'dy to pass ', 'f certain d', 'at it will ', 'e convince ', 'ent told hi', 'ampaign and', 'rver side s', 'ious texts ', 'o capitaliz', 'a duplicate', 'gh ann es d

In [8]:
def logprob(predictions, labels):
  """Log-probability of the true labels in a predicted batch."""
  predictions[predictions < 1e-10] = 1e-10
  return np.sum(np.multiply(labels, -np.log(predictions))) / labels.shape[0]

def sample_distribution(distribution):
  """Sample one element from a distribution assumed to be an array of normalized
  probabilities.
  """
  r = random.uniform(0, 1)
  s = 0
  for i in range(len(distribution)):
    s += distribution[i]
    if s >= r:
      return i
  return len(distribution) - 1

def sample(prediction):
  """Turn a (column) prediction into 1-hot encoded samples."""
  p = np.zeros(shape=[1, vocabulary_size], dtype=np.float)
  p[0, sample_distribution(prediction[0])] = 1.0
  return p

def random_distribution():
  """Generate a random column of probabilities."""
  b = np.random.uniform(0.0, 1.0, size=[1, vocabulary_size])
  return b/np.sum(b, 1)[:,None]

In [9]:
# ignore - just some test snippets
print('vocabulary_size: ', vocabulary_size)
b = np.random.uniform(0.0, 1.0, size=[1, vocabulary_size])
print(b)
print(np.sum(b, 1))
print(np.sum(b, 1)[:,None])
print(np.sum(b, 1)[:,np.newaxis])
# None is an alias for newaxis.
print(b/np.sum(b, 1)[:,None])

# backprop notes - https://colah.github.io/posts/2015-08-Backprop/

vocabulary_size:  27
[[ 0.85367616  0.01087899  0.94865049  0.95157496  0.62547508  0.55917674
   0.44672812  0.70827636  0.57683717  0.84041805  0.15882782  0.99325062
   0.43550561  0.48523218  0.82869128  0.74652244  0.46009007  0.22279565
   0.55081017  0.98626254  0.35485344  0.76596851  0.91010021  0.53316858
   0.1702687   0.2939602   0.76171522]]
[ 16.17971537]
[[ 16.17971537]]
[[ 16.17971537]]
[[ 0.05276212  0.00067238  0.05863209  0.05881284  0.03865798  0.03456036
   0.02761038  0.04377558  0.03565187  0.0519427   0.00981648  0.06138863
   0.02691677  0.02999016  0.05121791  0.0461394   0.02843623  0.01377006
   0.03404325  0.06095673  0.021932    0.04734128  0.05624946  0.0329529
   0.01052359  0.01816844  0.04707841]]


Simple LSTM Model.

In [20]:
num_nodes = 64

graph = tf.Graph()
with graph.as_default():
  
  # Parameters:
  # Input gate: input, previous output, and bias.
  ix = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  im = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  ib = tf.Variable(tf.zeros([1, num_nodes]))
  # Forget gate: input, previous output, and bias.
  fx = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  fm = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  fb = tf.Variable(tf.zeros([1, num_nodes]))
  # Memory cell: input, state and bias.                             
  cx = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  cm = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  cb = tf.Variable(tf.zeros([1, num_nodes]))
  # Output gate: input, previous output, and bias.
  ox = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  om = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  ob = tf.Variable(tf.zeros([1, num_nodes]))
  # Variables saving state across unrollings.
  saved_output = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  saved_state = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  # Classifier weights and biases.
  w = tf.Variable(tf.truncated_normal([num_nodes, vocabulary_size], -0.1, 0.1))
  b = tf.Variable(tf.zeros([vocabulary_size]))
  
  # Definition of the cell computation.
  # TODO my understanding is 
  #    i=input, 
  #    o=previous output, which is h_t-1 in http://colah.github.io/posts/2015-08-Understanding-LSTMs/
  #    state=the current (previous) state
  def lstm_cell(i, o, state):
    """Create a LSTM cell. See e.g.: http://arxiv.org/pdf/1402.1128v1.pdf
    Note that in this formulation, we omit the various connections between the
    previous state and the gates."""
    # zzz see http://colah.github.io/posts/2015-08-Understanding-LSTMs/
    input_gate = tf.sigmoid(tf.matmul(i, ix) + tf.matmul(o, im) + ib)
    forget_gate = tf.sigmoid(tf.matmul(i, fx) + tf.matmul(o, fm) + fb)
    update = tf.matmul(i, cx) + tf.matmul(o, cm) + cb
    state = forget_gate * state + input_gate * tf.tanh(update)
    output_gate = tf.sigmoid(tf.matmul(i, ox) + tf.matmul(o, om) + ob)
    return output_gate * tf.tanh(state), state

  # Input data.
  train_data = list()
  for _ in range(num_unrollings + 1):
    train_data.append(
      tf.placeholder(tf.float32, shape=[batch_size,vocabulary_size]))
  print('len(train_data)', len(train_data))
  print('train_data[0].shape', train_data[0].shape)
  train_inputs = train_data[:num_unrollings]
  print('len(train_inputs)', len(train_inputs))
  train_labels = train_data[1:]  # labels are inputs shifted by one time step.
  print('len(train_labels)', len(train_labels))
  print('train_labels[0].shape', train_labels[0].shape)

  # Unrolled LSTM loop.
  outputs = list()
  output = saved_output
  state = saved_state
  for i in train_inputs:
    output, state = lstm_cell(i, output, state)
    outputs.append(output)
  print('len(outputs)', len(outputs))
  print('outputs[0].shape', outputs[0].shape)
  print('tf.concat(outputs, 0).shape', tf.concat(outputs, 0).shape)
  print('w.get_shape()', w.get_shape())
  print('b.get_shape()', b.get_shape())
  print('numpy broadcasting allows adding (640,27) + (27,1):')
  print ('tf.nn.xw_plus_b(tf.concat(outputs, 0), w, b).shape', tf.nn.xw_plus_b(tf.concat(outputs, 0), w, b).shape)

  # State saving across unrollings.
  with tf.control_dependencies([saved_output.assign(output),
                                saved_state.assign(state)]):
    # Classifier.
    # 'tf.concat(outputs, 0)' will create a matrix of shape (10*64,64)=(640,64)
    # w is (64,27). b is (27,1)
    # addition is via numpy broadcasting
    logits = tf.nn.xw_plus_b(tf.concat(outputs, 0), w, b)
    loss = tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits(
        labels=tf.concat(train_labels, 0), logits=logits))

  # Optimizer.
  global_step = tf.Variable(0)
  learning_rate = tf.train.exponential_decay(
    10.0, global_step, 5000, 0.1, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate)
  gradients, v = zip(*optimizer.compute_gradients(loss))
  gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
  optimizer = optimizer.apply_gradients(
    zip(gradients, v), global_step=global_step)

  # Predictions.
  train_prediction = tf.nn.softmax(logits)
  
  # Sampling and validation eval: batch 1, no unrolling.
  sample_input = tf.placeholder(tf.float32, shape=[1, vocabulary_size])
  saved_sample_output = tf.Variable(tf.zeros([1, num_nodes]))
  saved_sample_state = tf.Variable(tf.zeros([1, num_nodes]))
  reset_sample_state = tf.group(
    saved_sample_output.assign(tf.zeros([1, num_nodes])),
    saved_sample_state.assign(tf.zeros([1, num_nodes])))
  sample_output, sample_state = lstm_cell(
    sample_input, saved_sample_output, saved_sample_state)
  with tf.control_dependencies([saved_sample_output.assign(sample_output),
                                saved_sample_state.assign(sample_state)]):
    sample_prediction = tf.nn.softmax(tf.nn.xw_plus_b(sample_output, w, b))

len(train_data) 11
train_data[0].shape (64, 27)
len(train_inputs) 10
len(train_labels) 10
train_labels[0].shape (64, 27)
len(outputs) 10
outputs[0].shape (64, 64)
tf.concat(outputs, 0).shape (640, 64)
w.get_shape() (64, 27)
b.get_shape() (27,)
numpy broadcasting allows adding (640,27) + (27,1):
tf.nn.xw_plus_b(tf.concat(outputs, 0), w, b).shape (640, 27)


In [21]:
num_steps = 7001
summary_frequency = 100

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  mean_loss = 0
  for step in range(num_steps):
    batches = train_batches.next()
    feed_dict = dict()
    for i in range(num_unrollings + 1):
      feed_dict[train_data[i]] = batches[i]
    _, l, predictions, lr = session.run(
      [optimizer, loss, train_prediction, learning_rate], feed_dict=feed_dict)
    mean_loss += l
    if step % summary_frequency == 0:
      if step > 0:
        mean_loss = mean_loss / summary_frequency
      # The mean loss is an estimate of the loss over the last few batches.
      print(
        'Average loss at step %d: %f learning rate: %f' % (step, mean_loss, lr))
      mean_loss = 0
      labels = np.concatenate(list(batches)[1:])
      print('Minibatch perplexity: %.2f' % float(
        np.exp(logprob(predictions, labels))))
      if step % (summary_frequency * 10) == 0:
        # Generate some samples.
        print('=' * 80)
        for _ in range(5):
          feed = sample(random_distribution())
          sentence = characters(feed)[0]
          reset_sample_state.run()
          for _ in range(79):
            prediction = sample_prediction.eval({sample_input: feed})
            feed = sample(prediction)
            sentence += characters(feed)[0]
          print(sentence)
        print('=' * 80)
      # Measure validation set perplexity.
      reset_sample_state.run()
      valid_logprob = 0
      for _ in range(valid_size):
        b = valid_batches.next()
        predictions = sample_prediction.eval({sample_input: b[0]})
        valid_logprob = valid_logprob + logprob(predictions, b[1])
      print('Validation set perplexity: %.2f' % float(np.exp(
        valid_logprob / valid_size)))

Initialized
Average loss at step 0: 3.299718 learning rate: 10.000000
Minibatch perplexity: 27.11
hfci cng nairdxao sdzacokxszvhnulhuecefrcsah vgfe dwoexcnrigdzt hfnndeeheaoyre t
c hrqs oxlyylvaj grtf toqinopnczocedh btbureh cthvetbmidgqnhrwntr aym tartqlpsi 
mmyzejik oe aovkip ccdaazqkqa xiqcemqhw akga drhvrbal xbophfestxvdoefdysnhhedgle
nrpedfw hv arnu zueariubhkwsrtoxrwlhyevruzztwbgetckeaxtlrg s qux e rfradwzr gagk
ap  ntetyaotlbhosrxsl vqdphamom v mju sw ihnvuscsa rpw tge trxm vkd nohldoktrilp
Validation set perplexity: 20.19
Average loss at step 100: 2.591237 learning rate: 10.000000
Minibatch perplexity: 10.84
Validation set perplexity: 10.20
Average loss at step 200: 2.240612 learning rate: 10.000000
Minibatch perplexity: 8.55
Validation set perplexity: 8.48
Average loss at step 300: 2.098266 learning rate: 10.000000
Minibatch perplexity: 7.58
Validation set perplexity: 7.78
Average loss at step 400: 2.005198 learning rate: 10.000000
Minibatch perplexity: 7.47
Validation set per

---
Problem 1
---------

You might have noticed that the definition of the LSTM cell involves 4 matrix multiplications with the input, and 4 matrix multiplications with the output. Simplify the expression by using a single matrix multiply for each, and variables that are 4 times larger.

---

In [23]:
num_nodes = 64

graph = tf.Graph()
with graph.as_default():
  
  # Parameters:
  # Input gate: input, previous output, and bias.
  ix = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  im = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  ib = tf.Variable(tf.zeros([1, num_nodes]))
  # Forget gate: input, previous output, and bias.
  fx = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  fm = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  fb = tf.Variable(tf.zeros([1, num_nodes]))
  # Memory cell: input, state and bias.                             
  cx = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  cm = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  cb = tf.Variable(tf.zeros([1, num_nodes]))
  # Output gate: input, previous output, and bias.
  ox = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  om = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  ob = tf.Variable(tf.zeros([1, num_nodes]))
  # Variables saving state across unrollings.
  saved_output = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  saved_state = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  # Classifier weights and biases.
  w = tf.Variable(tf.truncated_normal([num_nodes, vocabulary_size], -0.1, 0.1))
  b = tf.Variable(tf.zeros([vocabulary_size]))
    
  # Maybe use tf.concat here to combine the existing definitions instead of creating new ones?
  ix_fx_cx_ox = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes*4], -0.1, 0.1))
  im_fm_cm_om = tf.Variable(tf.truncated_normal([num_nodes, num_nodes*4], -0.1, 0.1))
  ib_fb_cb_ob = tf.Variable(tf.zeros([1, num_nodes*4]))
  
  # Definition of the cell computation.
  # TODO my understanding is 
  #    i=input, 
  #    o=previous output, which is h_t-1 in http://colah.github.io/posts/2015-08-Understanding-LSTMs/
  #    state=the current (previous) state
  def lstm_cell(i, o, state):
    """Create a LSTM cell. See e.g.: http://arxiv.org/pdf/1402.1128v1.pdf
    Note that in this formulation, we omit the various connections between the
    previous state and the gates."""
    # ------------ start of original definition ---------------
    # zzz see http://colah.github.io/posts/2015-08-Understanding-LSTMs/
#     input_gate = tf.sigmoid(tf.matmul(i, ix) + tf.matmul(o, im) + ib)
#     forget_gate = tf.sigmoid(tf.matmul(i, fx) + tf.matmul(o, fm) + fb)
#     update = tf.matmul(i, cx) + tf.matmul(o, cm) + cb
#     state = forget_gate * state + input_gate * tf.tanh(update)
#     output_gate = tf.sigmoid(tf.matmul(i, ox) + tf.matmul(o, om) + ob)
#     return output_gate * tf.tanh(state), state
    # ------------- end of original definition ----------------

    # The horizontally-concatenated matrix mults and additions can all be done at once here.
    # Basically we are doing these four things at once:
    #    tf.matmul(i, ix) + tf.matmul(o, im) + ib
    #    tf.matmul(i, fx) + tf.matmul(o, fm) + fb
    #    tf.matmul(i, cx) + tf.matmul(o, cm) + cb
    #    tf.matmul(i, ox) + tf.matmul(o, om) + ob
    # We are doing this at once, where ix_fx_cx_ox has shape (vocabulary_size, num_nodes*4), and
    # im_fm_cm_om has shape (num_nodes, num_nodes*4)
    # and ib_fb_cb_ob has shape (1, num_nodes*4)
    matrix_calc_output = tf.matmul(i, ix_fx_cx_ox) + tf.matmul(o, im_fm_cm_om) + ib_fb_cb_ob
    input_raw, forget_raw, update_raw, output_raw = tf.split(matrix_calc_output, num_or_size_splits=4, axis=1)
    input_gate = tf.sigmoid(input_raw)
    forget_gate = tf.sigmoid(forget_raw)
    update = update_raw # No need to do anything else for update
    output_gate = tf.sigmoid(output_raw)
    state = forget_gate * state + input_gate * tf.tanh(update) # this line is unchanged
    return output_gate * tf.tanh(state), state # this line us unchanged
    

  # Input data.
  train_data = list()
  for _ in range(num_unrollings + 1):
    train_data.append(
      tf.placeholder(tf.float32, shape=[batch_size,vocabulary_size]))
  print('len(train_data)', len(train_data))
  print('train_data[0].shape', train_data[0].shape)
  train_inputs = train_data[:num_unrollings]
  print('len(train_inputs)', len(train_inputs))
  train_labels = train_data[1:]  # labels are inputs shifted by one time step.
  print('len(train_labels)', len(train_labels))
  print('train_labels[0].shape', train_labels[0].shape)

  # Unrolled LSTM loop.
  outputs = list()
  output = saved_output
  state = saved_state
  for i in train_inputs:
    output, state = lstm_cell(i, output, state)
    outputs.append(output)
  print('len(outputs)', len(outputs))
  print('outputs[0].shape', outputs[0].shape)
  print('tf.concat(outputs, 0).shape', tf.concat(outputs, 0).shape)
  print('w.get_shape()', w.get_shape())
  print('b.get_shape()', b.get_shape())
  print('numpy broadcasting allows adding (640,27) + (27,1):')
  print ('tf.nn.xw_plus_b(tf.concat(outputs, 0), w, b).shape', tf.nn.xw_plus_b(tf.concat(outputs, 0), w, b).shape)

  # State saving across unrollings.
  with tf.control_dependencies([saved_output.assign(output),
                                saved_state.assign(state)]):
    # Classifier.
    # 'tf.concat(outputs, 0)' will create a matrix of shape (10*64,64)=(640,64)
    # w is (64,27). b is (27,1)
    # addition is via numpy broadcasting
    logits = tf.nn.xw_plus_b(tf.concat(outputs, 0), w, b)
    loss = tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits(
        labels=tf.concat(train_labels, 0), logits=logits))

  # Optimizer.
  global_step = tf.Variable(0)
  learning_rate = tf.train.exponential_decay(
    10.0, global_step, 5000, 0.1, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate)
  gradients, v = zip(*optimizer.compute_gradients(loss))
  gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
  optimizer = optimizer.apply_gradients(
    zip(gradients, v), global_step=global_step)

  # Predictions.
  train_prediction = tf.nn.softmax(logits)
  
  # Sampling and validation eval: batch 1, no unrolling.
  sample_input = tf.placeholder(tf.float32, shape=[1, vocabulary_size])
  saved_sample_output = tf.Variable(tf.zeros([1, num_nodes]))
  saved_sample_state = tf.Variable(tf.zeros([1, num_nodes]))
  reset_sample_state = tf.group(
    saved_sample_output.assign(tf.zeros([1, num_nodes])),
    saved_sample_state.assign(tf.zeros([1, num_nodes])))
  sample_output, sample_state = lstm_cell(
    sample_input, saved_sample_output, saved_sample_state)
  with tf.control_dependencies([saved_sample_output.assign(sample_output),
                                saved_sample_state.assign(sample_state)]):
    sample_prediction = tf.nn.softmax(tf.nn.xw_plus_b(sample_output, w, b))

len(train_data) 11
train_data[0].shape (64, 27)
len(train_inputs) 10
len(train_labels) 10
train_labels[0].shape (64, 27)
len(outputs) 10
outputs[0].shape (64, 64)
tf.concat(outputs, 0).shape (640, 64)
w.get_shape() (64, 27)
b.get_shape() (27,)
numpy broadcasting allows adding (640,27) + (27,1):
tf.nn.xw_plus_b(tf.concat(outputs, 0), w, b).shape (640, 27)


In [24]:
num_steps = 7001
summary_frequency = 100

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  mean_loss = 0
  for step in range(num_steps):
    batches = train_batches.next()
    feed_dict = dict()
    for i in range(num_unrollings + 1):
      feed_dict[train_data[i]] = batches[i]
    _, l, predictions, lr = session.run(
      [optimizer, loss, train_prediction, learning_rate], feed_dict=feed_dict)
    mean_loss += l
    if step % summary_frequency == 0:
      if step > 0:
        mean_loss = mean_loss / summary_frequency
      # The mean loss is an estimate of the loss over the last few batches.
      print(
        'Average loss at step %d: %f learning rate: %f' % (step, mean_loss, lr))
      mean_loss = 0
      labels = np.concatenate(list(batches)[1:])
      print('Minibatch perplexity: %.2f' % float(
        np.exp(logprob(predictions, labels))))
      if step % (summary_frequency * 10) == 0:
        # Generate some samples.
        print('=' * 80)
        for _ in range(5):
          feed = sample(random_distribution())
          sentence = characters(feed)[0]
          reset_sample_state.run()
          for _ in range(79):
            prediction = sample_prediction.eval({sample_input: feed})
            feed = sample(prediction)
            sentence += characters(feed)[0]
          print(sentence)
        print('=' * 80)
      # Measure validation set perplexity.
      reset_sample_state.run()
      valid_logprob = 0
      for _ in range(valid_size):
        b = valid_batches.next()
        predictions = sample_prediction.eval({sample_input: b[0]})
        valid_logprob = valid_logprob + logprob(predictions, b[1])
      print('Validation set perplexity: %.2f' % float(np.exp(
        valid_logprob / valid_size)))

Initialized
Average loss at step 0: 3.293252 learning rate: 10.000000
Minibatch perplexity: 26.93
cguf epfxdg   djkefme zn ga tnirrmxu  eramohr uzalne exevozwwyijees owr iatoikoy
xorkvk  zcib a id gz qaahuezauhces lfxjetqisalnae   ixsmayayhbcbbrreeum euy  kqw
epdhheavjgm m e e aabb ce itf agfiabv etjsoath p  sj ptqmawqbw  stcrnsodmoopmzyo
ornqu dai m edgje a enihlwe cjw uz rkt eesxcsz ezvaucvjmor fxknkaxiryjqjqareeaa 
gb lxsyhrye ordslyi   fyawhp s gjiam eeugsdica aseeh eynhpvt gbreimtre hetddjeul
Validation set perplexity: 20.06
Average loss at step 100: 2.582779 learning rate: 10.000000
Minibatch perplexity: 10.45
Validation set perplexity: 10.39
Average loss at step 200: 2.247702 learning rate: 10.000000
Minibatch perplexity: 8.46
Validation set perplexity: 8.92
Average loss at step 300: 2.088998 learning rate: 10.000000
Minibatch perplexity: 6.25
Validation set perplexity: 8.04
Average loss at step 400: 2.037878 learning rate: 10.000000
Minibatch perplexity: 7.88
Validation set per

---
Problem 2
---------

We want to train a LSTM over bigrams, that is pairs of consecutive characters like 'ab' instead of single characters like 'a'. Since the number of possible bigrams is large, feeding them directly to the LSTM using 1-hot encodings will lead to a very sparse representation that is very wasteful computationally.

a- Introduce an embedding lookup on the inputs, and feed the embeddings to the LSTM cell instead of the inputs themselves.

b- Write a bigram-based LSTM, modeled on the character LSTM above.

c- Introduce Dropout. For best practices on how to use Dropout in LSTMs, refer to this [article](http://arxiv.org/abs/1409.2329).

---

---
Problem 3
---------

(difficult!)

Write a sequence-to-sequence LSTM which mirrors all the words in a sentence. For example, if your input is:

    the quick brown fox
    
the model should attempt to output:

    eht kciuq nworb xof
    
Refer to the lecture on how to put together a sequence-to-sequence model, as well as [this article](http://arxiv.org/abs/1409.3215) for best practices.

---